In [2]:
#!/usr/bin/env Python

from collections import defaultdict, deque
import uuid
from string import ascii_lowercase as alphabet
import re
from hashlib import md5
import pickle as pickle
import alchemy
from textdistance import jaro_winkler
from collections import Counter
#from alchemy import get_config, match
from alchemy import match
from alchemy.schema import *
from alchemy.match import commit_inserts, commit_updates
from handlers.xml_util import normalize_utf8
from datetime import datetime
from sqlalchemy.sql import or_
from sqlalchemy.sql.expression import bindparam
from unidecode import unidecode
from tasks import bulk_commit_inserts, bulk_commit_updates
import sys
import os
import configparser
import tqdm
project_home=os.environ["PACKAGE_HOME"]
project_home = os.environ['PACKAGE_HOME']
config = configparser.ConfigParser()
config.read(project_home + '/Development/config.ini')
THRESHOLD = config["LAWYER"]["THRESHOLD"]


nodigits = re.compile(r'[^\d]+')

disambig_folder = '{}/{}'.format(config['FOLDERS']['WORKING_FOLDER'],'update_20191008')
disambig_folder

'/dbupdate_data//update_20191008'

In [3]:

# def get_lawyer_id(obj):
#     """
#     Returns string representing an lawyer object. Returns obj.organization if
#     it exists, else returns concatenated obj.name_first + '|' + obj.name_last
#     """
#     if obj.organization:
#         return obj.organization
#     try:
#         return obj.name_first + '|' + obj.name_last
#     except:
#         return ''

# def clean_lawyers(list_of_lawyers):
#     """
#     Removes the following stop words from each lawyer:
#     the, of, and, a, an, at
#     Then, blocks the lawyer with other lawyers that start
#     with the same letter. Returns a list of these blocks
#     """
#     stoplist = ['the', 'of', 'and', 'a', 'an', 'at']
#     #alpha_blocks = defaultdict(list)
#     block = []
#     #print('Removing stop words, blocking by first letter...', flush=True)
#     for lawyer in list_of_lawyers:
#         lawyer_dict[lawyer.uuid] = lawyer
#         a_id = get_lawyer_id(lawyer)
#         # removes stop words, then rejoins the string
#         a_id = ' '.join([x for x in a_id.split(' ') if x.lower() not in stoplist])
#         a_id = ''.join(nodigits.findall(a_id)).strip()
#         id_map[a_id].append(lawyer.uuid)
#         block.append(a_id)
#     print('lawyers cleaned!', flush=True)
#     return block


def without_digits(word):
    return ''.join([x for x in word if not x.isdigit()])

def create_jw_blocks(list_of_lawyers):
    """
    Receives list of blocks, where a block is a list of lawyers
    that all begin with the same letter. Within each block, does
    a pairwise jaro winkler comparison to block lawyers together
    """
    global blocks
    consumed = defaultdict(int)
    print('Doing pairwise Jaro-Winkler...', len(list_of_lawyers), flush=True)
    for i, primary in enumerate(list_of_lawyers):
        if consumed[primary]: continue
        consumed[primary] = 1
        blocks[primary].append(primary)
        for secondary in list_of_lawyers[i:]:
            if consumed[secondary]: continue
            if primary == secondary:
                blocks[primary].append(secondary)
                continue
            if jaro_winkler(primary, secondary, 0.0) >= float(THRESHOLD):
                consumed[secondary] = 1
                blocks[primary].append(secondary)
    pickle.dump(blocks, open('lawyer.pickle', 'wb'))
    print('lawyer blocks created!', flush=True)


lawyer_insert_statements = []
patentlawyer_insert_statements = []
update_statements = []
def create_lawyer_table(session):
    """
    Given a list of lawyers and the redis key-value disambiguation,
    populates the lawyer table in the database
    """
    print('Disambiguating lawyers...', flush=True)
    session.execute('set foreign_key_checks = 0;')
    session.commit()
    i = 0
    for lawyer in blocks.keys():
        ra_ids = (id_map[ra] for ra in blocks[lawyer])
        for block in ra_ids:
            i += 1
            rawlawyers = [lawyer_dict[ra_id] for ra_id in block]
            if i % 20000 == 0:
                print(i, datetime.now(), flush=True)
                lawyer_match(rawlawyers, session, commit=True)
            else:
                lawyer_match(rawlawyers, session, commit=False)
    t1 = bulk_commit_inserts(lawyer_insert_statements, Lawyer.__table__, 20000, 'grant')
    t2 = bulk_commit_inserts(patentlawyer_insert_statements, patentlawyer,  20000)
    t3 = bulk_commit_updates('lawyer_id', update_statements, RawLawyer.__table__, 20000)
    # t1.get()
    # t2.get()
    # t3.get()
    # session.commit()
    print(i, datetime.now(), flush=True)

def lawyer_match(objects, session, commit=False):
    freq = defaultdict(Counter)
    param = {}
    raw_objects = []
    clean_objects = []
    clean_cnt = 0
    clean_main = None
    class_type = None
    class_type = None
    for obj in objects:
        if not obj: continue
        class_type = obj.__related__
        raw_objects.append(obj)
        break

    param = {}
    for obj in raw_objects:
        for k, v in obj.summarize.items():
            freq[k][v] += 1
        if "id" not in param:
            param["id"] = obj.uuid
        param["id"] = min(param["id"], obj.uuid)

    # create parameters based on most frequent
    for k in freq:
        if None in freq[k]:
            freq[k].pop(None)
        if "" in freq[k]:
            freq[k].pop("")
        if freq[k]:
            param[k] = freq[k].most_common(1)[0][0]
    if 'organization' not in param:
        param['organization'] = ''
    if 'type' not in param:
        param['type'] = ''
    if 'name_last' not in param:
        param['name_last'] = ''
    if 'name_first' not in param:
        param['name_first'] = ''
    if 'residence' not in param:
        param['residence'] = ''
    if 'nationality' not in param:
        param['nationality'] = ''
    if 'country' not in param:
        param['country'] = ''

    if param["organization"]:
        param["id"] = md5(unidecode(param["organization"]).encode('utf-8')).hexdigest()
    if param["name_last"]:
        param["id"] = md5(unidecode(param["name_last"]+param["name_first"]).encode('utf-8')).hexdigest()
    
    lawyer_insert_statements.append(param)
    tmpids = [x.uuid for x in objects]
    patents = [x.patent_id for x in objects]
    patentlawyer_insert_statements.extend([{'patent_id': x, 'lawyer_id': param['id']} for x in patents])
    update_statements.extend([{'pk':x,'update':param['id']} for x in tmpids])

def examine():
    lawyers = s.query(lawyer).all()
    for a in lawyers:
        print(get_lawyer_id(a), len(a.rawlawyers), flush=True)
        for ra in a.rawlawyers:
            if get_lawyer_id(ra) != get_lawyer_id(a):
                print(get_lawyer_id(ra), flush=True)
            print('-'*10, flush=True)
    print(len(lawyers), flush=True)


def printall():
    lawyers = s.query(lawyer).all()
    with open('out.txt', 'wb') as f:
        for a in lawyers:
            f.write(normalize_utf8(get_lawyer_id(a)).encode('utf-8'))
            f.write('\n')
            for ra in a.rawlawyers:
                f.write(normalize_utf8(get_lawyer_id(ra)).encode('utf-8'))
                f.write('\n')
            f.write('-'*20)
            f.write('\n')


def run_letter(letter, session, doctype='grant'):
    schema = RawLawyer
    if doctype == 'application':
        schema = App_RawLawyer
    letter = letter.upper()
    clause1 = schema.organization.startswith(bindparam('letter',letter))
    clause2 = schema.name_first.startswith(bindparam('letter',letter))
    clauses = or_(clause1, clause2)
    lawyers = (lawyer for lawyer in session.query(schema).filter(clauses))
    block = clean_lawyers(lawyers)
    create_jw_blocks(block)
    create_lawyer_table(session)



In [4]:
# re-written 
def run_disambiguation(doctype='grant'):
    # get all lawyers in database
    global blocks
    global lawyer_insert_statements
    global patentlawyer_insert_statements
    global update_statements
    session = alchemy.fetch_session(dbtype=doctype)
    
    # bookkeeping for blocks
    blocks = defaultdict(list)
    id_map = defaultdict(list)
    lawyer_dict = {}
    
    print("going through alphabet")
    for letter in alphabet:
       
        print("letter is: ", letter, datetime.now(), flush=True)

        blocks = defaultdict(list)
        lawyer_insert_statements = []
        patentlawyer_insert_statements = []
        update_statements = []

        # query by letter 
        lawyers_object = session.query(RawLawyer).add_columns(RawLawyer.uuid, RawLawyer.lawyer_id, RawLawyer.cleaned_alpha_lawyer_id)#.filter(RawLawyer.cleaned_alpha_lawyer_id like (letter + '%'))

        letterblock = []
        # update lawyer_dict, id_map, and letterblock
        for lawyer in lawyers_object:
            lawyer_dict[lawyer.uuid] = lawyer
            id_map[lawyer.cleaned_alpha_lawyer_id].append(lawyer.uuid)
            letterblock.append(lawyer.cleaned_alpha_lawyer_id)

        create_jw_blocks(letterblock)
        create_lawyer_table(session)
    

In [8]:
if __name__ == '__main__':

    print("running")
    doctype='grant' 
    global blocks
    global lawyer_insert_statements
    global patentlawyer_insert_statements
    global update_statements

   
    session = alchemy.fetch_session(dbtype=doctype) 
    
    with open(disambig_folder + '/rawlawyer_memory_log.txt', 'w') as f:
        
    
        print("going through alphabet")
        for letter in alphabet:

            # track memory usage by letter
            f.write('letter is ' + letter + '\n')

            print("letter is: ", letter, datetime.now(), flush=True)

            # bookkeeping
            id_map = defaultdict(list)
            lawyer_dict = {}
            letterblock = []
            blocks = defaultdict(list)
            lawyer_insert_statements = []
            patentlawyer_insert_statements = []
            update_statements = []

            # query by letter        
            lawyers_object = session.query(RawLawyer).filter(RawLawyer.cleaned_alpha_lawyer_id.like(letter + '%'))

            print("query returned")
            
            f.write('size of lawyers_object for letter is ' + str(sys.getsizeof(lawyers_object)) + '\n')

            for lawyer in lawyers_object:
                lawyer_dict[lawyer.uuid] = lawyer
                id_map[lawyer.cleaned_alpha_lawyer_id].append(lawyer.uuid)
                letterblock.append(lawyer.cleaned_alpha_lawyer_id)

                
                
            f.write('size of lawyer_dict for letter is ' + str(sys.getsizeof(lawyer_dict)) + '\n')
            f.write('size of id_map for letter is ' + str(sys.getsizeof(id_map)) + '\n')
            f.write('size of letterblock for letter is ' + str(sys.getsizeof(letterblock)) + '\n')
            
            print(letterblock[0:5])
            create_jw_blocks(letterblock)
            
            f.write('size of blocks for letter is ' + str(sys.getsizeof(blocks)) + '\n')
            create_lawyer_table(session)
            
            f.write('size of lawyer_insert_statements for letter is ' + str(sys.getsizeof(lawyer_insert_statements)) + '\n')
            f.write('size of patentlawyer_insert_statements for letter is ' + str(sys.getsizeof(patentlawyer_insert_statements)) + '\n')
            f.write('size of update_statements for letter is ' + str(sys.getsizeof(update_statements)) + '\n')
            
            f.write('----------------------------------------------' + '\n')
            f.flush()
            
    f.close()



running
going through alphabet
letter is:  a 2019-11-01 15:32:36.367866
query returned
['Alan J.|Hickman', 'Anthony R.|Lambert', 'Alston & Birds LLP', 'Anthony J.|Casella', 'Antonelli, Terry & Wands']
Doing pairwise Jaro-Winkler... 364269
lawyer blocks created!
Disambiguating lawyers...
committing chunk 1 of 8 with length 1000 at 2019-11-01 15:40:26.361166
committing chunk 2 of 8 with length 1000 at 2019-11-01 15:40:26.407372
committing chunk 3 of 8 with length 1000 at 2019-11-01 15:40:26.454348
committing chunk 4 of 8 with length 1000 at 2019-11-01 15:40:26.498898
committing chunk 5 of 8 with length 1000 at 2019-11-01 15:40:26.543150
committing chunk 6 of 8 with length 1000 at 2019-11-01 15:40:26.609858
committing chunk 7 of 8 with length 1000 at 2019-11-01 15:40:26.680765
committing chunk 8 of 8 with length 1000 at 2019-11-01 15:40:26.732148
committing last 625 records at 2019-11-01 15:40:26.738692
last chunk in, committing
committing chunk 1 of 364 with length 1000 at 2019-11-01 15:

committing chunk 100 of 364 with length 1000 at 2019-11-01 15:40:32.235760
committing chunk 101 of 364 with length 1000 at 2019-11-01 15:40:32.283868
committing chunk 102 of 364 with length 1000 at 2019-11-01 15:40:32.358994
committing chunk 103 of 364 with length 1000 at 2019-11-01 15:40:32.407240
committing chunk 104 of 364 with length 1000 at 2019-11-01 15:40:32.548357
committing chunk 105 of 364 with length 1000 at 2019-11-01 15:40:32.829998
committing chunk 106 of 364 with length 1000 at 2019-11-01 15:40:32.883486
committing chunk 107 of 364 with length 1000 at 2019-11-01 15:40:32.928842
committing chunk 108 of 364 with length 1000 at 2019-11-01 15:40:32.976056
committing chunk 109 of 364 with length 1000 at 2019-11-01 15:40:33.027371
committing chunk 110 of 364 with length 1000 at 2019-11-01 15:40:33.076465
committing chunk 111 of 364 with length 1000 at 2019-11-01 15:40:33.123808
committing chunk 112 of 364 with length 1000 at 2019-11-01 15:40:33.170575
committing chunk 113 of 3

committing chunk 211 of 364 with length 1000 at 2019-11-01 15:40:42.143220
committing chunk 212 of 364 with length 1000 at 2019-11-01 15:40:42.189808
committing chunk 213 of 364 with length 1000 at 2019-11-01 15:40:42.234766
committing chunk 214 of 364 with length 1000 at 2019-11-01 15:40:42.287420
committing chunk 215 of 364 with length 1000 at 2019-11-01 15:40:42.379388
committing chunk 216 of 364 with length 1000 at 2019-11-01 15:40:42.456301
committing chunk 217 of 364 with length 1000 at 2019-11-01 15:40:42.502298
committing chunk 218 of 364 with length 1000 at 2019-11-01 15:40:42.548534
committing chunk 219 of 364 with length 1000 at 2019-11-01 15:40:42.606719
committing chunk 220 of 364 with length 1000 at 2019-11-01 15:40:42.665134
committing chunk 221 of 364 with length 1000 at 2019-11-01 15:40:42.776563
committing chunk 222 of 364 with length 1000 at 2019-11-01 15:40:42.835025
committing chunk 223 of 364 with length 1000 at 2019-11-01 15:40:42.892405
committing chunk 224 of 3

committing chunk 321 of 364 with length 1000 at 2019-11-01 15:40:50.928341
committing chunk 322 of 364 with length 1000 at 2019-11-01 15:40:50.980037
committing chunk 323 of 364 with length 1000 at 2019-11-01 15:40:51.037833
committing chunk 324 of 364 with length 1000 at 2019-11-01 15:40:51.089007
committing chunk 325 of 364 with length 1000 at 2019-11-01 15:40:51.136021
committing chunk 326 of 364 with length 1000 at 2019-11-01 15:40:51.183540
committing chunk 327 of 364 with length 1000 at 2019-11-01 15:40:51.231624
committing chunk 328 of 364 with length 1000 at 2019-11-01 15:40:51.280894
committing chunk 329 of 364 with length 1000 at 2019-11-01 15:40:51.328190
committing chunk 330 of 364 with length 1000 at 2019-11-01 15:40:51.377158
committing chunk 331 of 364 with length 1000 at 2019-11-01 15:40:51.423683
committing chunk 332 of 364 with length 1000 at 2019-11-01 15:40:51.516431
committing chunk 333 of 364 with length 1000 at 2019-11-01 15:40:51.648083
committing chunk 334 of 3

committing chunk 67 of 364 with length 1000 at 2019-11-01 15:40:56.673174
committing chunk 68 of 364 with length 1000 at 2019-11-01 15:40:56.710294
committing chunk 69 of 364 with length 1000 at 2019-11-01 15:40:56.748638
committing chunk 70 of 364 with length 1000 at 2019-11-01 15:40:56.790187
committing chunk 71 of 364 with length 1000 at 2019-11-01 15:40:56.826284
committing chunk 72 of 364 with length 1000 at 2019-11-01 15:40:56.868243
committing chunk 73 of 364 with length 1000 at 2019-11-01 15:40:56.923151
committing chunk 74 of 364 with length 1000 at 2019-11-01 15:40:56.996466
committing chunk 75 of 364 with length 1000 at 2019-11-01 15:40:57.044782
committing chunk 76 of 364 with length 1000 at 2019-11-01 15:40:57.092477
committing chunk 77 of 364 with length 1000 at 2019-11-01 15:40:57.132559
committing chunk 78 of 364 with length 1000 at 2019-11-01 15:40:57.169251
committing chunk 79 of 364 with length 1000 at 2019-11-01 15:40:57.206823
committing chunk 80 of 364 with length

...

committing chunk 138 of 481 with length 1000 at 2019-11-01 20:26:27.296413
committing chunk 139 of 481 with length 1000 at 2019-11-01 20:26:27.368126
committing chunk 140 of 481 with length 1000 at 2019-11-01 20:26:27.408268
committing chunk 141 of 481 with length 1000 at 2019-11-01 20:26:27.445627
committing chunk 142 of 481 with length 1000 at 2019-11-01 20:26:27.482990
committing chunk 143 of 481 with length 1000 at 2019-11-01 20:26:27.521719
committing chunk 144 of 481 with length 1000 at 2019-11-01 20:26:27.560484
committing chunk 145 of 481 with length 1000 at 2019-11-01 20:26:27.598643
committing chunk 146 of 481 with length 1000 at 2019-11-01 20:26:27.638876
committing chunk 147 of 481 with length 1000 at 2019-11-01 20:26:27.678459
committing chunk 148 of 481 with length 1000 at 2019-11-01 20:26:27.717543
committing chunk 149 of 481 with length 1000 at 2019-11-01 20:26:27.763339
committing chunk 150 of 481 with length 1000 at 2019-11-01 20:26:27.810834
committing chunk 151 of 4

committing chunk 250 of 481 with length 1000 at 2019-11-01 20:27:21.282343
committing chunk 251 of 481 with length 1000 at 2019-11-01 20:27:21.331179
committing chunk 252 of 481 with length 1000 at 2019-11-01 20:27:21.375357
committing chunk 253 of 481 with length 1000 at 2019-11-01 20:27:21.415618
committing chunk 254 of 481 with length 1000 at 2019-11-01 20:27:21.464988
committing chunk 255 of 481 with length 1000 at 2019-11-01 20:27:21.517463
committing chunk 256 of 481 with length 1000 at 2019-11-01 20:27:21.609109
committing chunk 257 of 481 with length 1000 at 2019-11-01 20:27:21.667274
committing chunk 258 of 481 with length 1000 at 2019-11-01 20:27:21.703832
committing chunk 259 of 481 with length 1000 at 2019-11-01 20:27:21.741932
committing chunk 260 of 481 with length 1000 at 2019-11-01 20:27:21.777785
committing chunk 261 of 481 with length 1000 at 2019-11-01 20:27:21.817564
committing chunk 262 of 481 with length 1000 at 2019-11-01 20:27:21.853313
committing chunk 263 of 4

committing chunk 362 of 481 with length 1000 at 2019-11-01 20:27:28.681743
committing chunk 363 of 481 with length 1000 at 2019-11-01 20:27:28.718859
committing chunk 364 of 481 with length 1000 at 2019-11-01 20:27:28.756782
committing chunk 365 of 481 with length 1000 at 2019-11-01 20:27:28.795676
committing chunk 366 of 481 with length 1000 at 2019-11-01 20:27:28.847898
committing chunk 367 of 481 with length 1000 at 2019-11-01 20:27:28.895797
committing chunk 368 of 481 with length 1000 at 2019-11-01 20:27:28.945858
committing chunk 369 of 481 with length 1000 at 2019-11-01 20:27:28.990582
committing chunk 370 of 481 with length 1000 at 2019-11-01 20:27:29.026764
committing chunk 371 of 481 with length 1000 at 2019-11-01 20:27:29.066629
committing chunk 372 of 481 with length 1000 at 2019-11-01 20:27:29.114775
committing chunk 373 of 481 with length 1000 at 2019-11-01 20:27:29.160351
committing chunk 374 of 481 with length 1000 at 2019-11-01 20:27:29.196493
committing chunk 375 of 4

committing chunk 476 of 481 with length 1000 at 2019-11-01 20:27:36.114993
committing chunk 477 of 481 with length 1000 at 2019-11-01 20:27:36.164261
committing chunk 478 of 481 with length 1000 at 2019-11-01 20:27:36.206029
committing chunk 479 of 481 with length 1000 at 2019-11-01 20:27:36.248231
committing chunk 480 of 481 with length 1000 at 2019-11-01 20:27:36.289293
committing chunk 481 of 481 with length 1000 at 2019-11-01 20:27:36.330630
committing last 934 records at 2019-11-01 20:27:36.340286
last chunk in, committing
8840 2019-11-01 20:27:55.960070
letter is:  x 2019-11-01 20:27:55.963448
query returned
['Xsensus, LLP', 'Xsensus LLP', 'Xsensus LLP', 'Xsensus LLP', 'Xiaodong|Li']
Doing pairwise Jaro-Winkler... 1681
lawyer blocks created!
Disambiguating lawyers...
committing last 81 records at 2019-11-01 20:28:03.808290
last chunk in, committing
committing chunk 1 of 1 with length 1000 at 2019-11-01 20:28:03.989490
committing last 681 records at 2019-11-01 20:28:03.995583
last

committing chunk 44 of 49 with length 1000 at 2019-11-01 20:28:22.816861
committing chunk 45 of 49 with length 1000 at 2019-11-01 20:28:22.852491
committing chunk 46 of 49 with length 1000 at 2019-11-01 20:28:22.897892
committing chunk 47 of 49 with length 1000 at 2019-11-01 20:28:22.943935
committing chunk 48 of 49 with length 1000 at 2019-11-01 20:28:22.994444
committing chunk 49 of 49 with length 1000 at 2019-11-01 20:28:23.035208
committing last 19 records at 2019-11-01 20:28:23.046139
last chunk in, committing
651 2019-11-01 20:28:24.211810
letter is:  z 2019-11-01 20:28:24.213380
query returned
['Zachary D.|Hadd', 'Zagorin Cave LLP', 'Zachary D.|Hadd', 'Zachary T.|Wobensmith, III', 'Zarley, McKee, Thomte, Voorhees & Sease']
Doing pairwise Jaro-Winkler... 17648
lawyer blocks created!
Disambiguating lawyers...
committing last 491 records at 2019-11-01 20:28:32.894996
last chunk in, committing
committing chunk 1 of 17 with length 1000 at 2019-11-01 20:28:33.094485
committing chunk 2